In [1]:
import pandas as pd

# Load the two datasets (replace 'ratings.csv' and 'wine_characteristics.csv' with your actual CSV file paths)
ratings_data = pd.read_csv('Dataset/last/XWines_Slim_150K_ratings.csv')
wine_data = pd.read_csv('Dataset/last/XWines_Slim_1K_wines.csv')
group_data = pd.read_csv('Dataset/last/group_composition.csv')  # Add path to your group CSV

# Merge the two datasets on 'WineID'
merged_data = pd.merge(ratings_data, wine_data, on='WineID')

# Function to find the best-rated wine or suggest a completely different one if rating < 4
def recommend_wine_for_user(user_id, merged_data):
    # Filter wines rated by the specific user
    user_wines = merged_data[merged_data['UserID'] == user_id]

    if user_wines.empty:
        return f"No wines found for user {user_id}.", None

    # Find the wine with the highest rating by the user
    best_rated_wine = user_wines.loc[user_wines['Rating'].idxmax()]

    # If the best rating is 4 or higher, return that wine
    if best_rated_wine['Rating'] >= 4:
        return best_rated_wine['WineID'], best_rated_wine['Rating']

    # If no wine has a rating of 4 or higher, find a completely different wine
    else:
        # Define characteristics to consider (adjust based on your dataset)
        characteristics = ['Type', 'Body']

        # Filter out wines that are similar to the one the user rated poorly
        different_wines = merged_data
        for char in characteristics:
            different_wines = different_wines[different_wines[char] != best_rated_wine[char]]

        # If there are still wines left, choose one randomly or based on rating
        if not different_wines.empty:
            recommended_wine = different_wines.sample().iloc[0]  # Sample one random different wine
            return recommended_wine['WineID'], None

        return f"No sufficiently different wines found for user {user_id}.", None

# Function to recommend wine for a group and output in a DataFrame
def recommend_wine_for_group(group_id, group_data, merged_data):
    # Get the group members from the group data
    group_info = group_data[group_data['group_id'] == group_id].iloc[0]
    group_members = eval(group_info['group_members'])  # Assuming group_members is a list stored as a string

    # Create a list to store each user's recommendation
    recommendations = []

    # Loop through each member of the group and get their favorite wine
    for user_id in group_members:
        wine_id, rating = recommend_wine_for_user(user_id, merged_data)
        recommendations.append({
            'group_id': group_id,
            'user_id': user_id,
            'wine_id': wine_id,
            'rating': rating if rating is not None else 'Suggested different wine'
        })

    # Convert the recommendations to a DataFrame
    recommendation_df = pd.DataFrame(recommendations)
    return recommendation_df

# Example usage for a specific group (replace with an actual group_id from your dataset)
group_id = 1  # Replace with the actual group_id you want to check
result_df = recommend_wine_for_group(group_id, group_data, merged_data)

# Output the DataFrame with the recommendations
print(result_df)


   group_id  user_id  wine_id  rating
0         1  1180876   111521     4.5
1         1  1006585   111422     5.0


/var/folders/qx/s0s669bd3mx83v921rrrxcjw0000gn/T/ipykernel_48204/3377510125.py:4: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  ratings_data = pd.read_csv('Dataset/last/XWines_Slim_150K_ratings.csv')


In [2]:

# Load the ratings dataset (replace 'ratings.csv' with your actual file path)
ratings_data = pd.read_csv('Dataset/last/XWines_Slim_150K_ratings.csv')

# Group by 'user_id' and find the maximum rating for each user
user_max_ratings = ratings_data.groupby('UserID')['Rating'].max()

# Filter to get only users whose max rating was 3
users_with_max_rating_3 = user_max_ratings[user_max_ratings == 3].index

# Print the list of users whose maximum rating was 3
print("Users whose maximum rating was 3:")
print(users_with_max_rating_3)


Users whose maximum rating was 3:
Index([1006724, 1056257, 1145862, 1196247, 1197650, 1220278, 1245717, 1279250,
       1290651, 1316369, 1334222, 1380716, 1391592, 1393496, 1397551, 1400386,
       1406797, 1420747, 1558571, 1563515, 1664756, 1719976, 1760544, 1762859,
       1781649, 1816088, 1843929, 1936578, 1944444, 1977712, 1997980, 2061195],
      dtype='int64', name='UserID')


/var/folders/qx/s0s669bd3mx83v921rrrxcjw0000gn/T/ipykernel_48204/744726888.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  ratings_data = pd.read_csv('Dataset/last/XWines_Slim_150K_ratings.csv')


In [3]:
# create average rating for each wine

ratings = {}
for index, row in ratings_data.iterrows():
    if row['WineID'] not in ratings:
        ratings[row['WineID']] = {"total": row['Rating'], "count": 1}
    else:
        ratings[row['WineID']]["total"] += row['Rating']
        ratings[row['WineID']]["count"] += 1

# add the averages to the df
wine_data["AvgRating"] = 0.0

for index, row in wine_data.iterrows():
    wine_data.loc[index, "AvgRating"] = ratings[row["WineID"]]["total"]/ratings[row["WineID"]]["count"]

In [4]:
wine_data

,WineID,WineName,Type,Elaborate,Grapes,Harmonize,ABV,Body,Acidity,Code,Country,RegionID,RegionName,WineryID,WineryName,Website,Vintages,AvgRating
0,100001,Espumante Moscatel,Sparkling,Varietal/100%,['Muscat/Moscato'],"['Pork', 'Rich Fish', 'Shellfish']",7.5,Medium-bodied,High,BR,Brazil,1001,Serra Gaúcha,10001,Casa Perini,http://www.vinicolaperini.com.br,"[2020, 2019, 2018, 2017, 2016, 2015, 2014, 201...",3.838462
1,100002,Ancellotta,Red,Varietal/100%,['Ancellotta'],"['Beef', 'Barbecue', 'Codfish', 'Pasta', 'Pizz...",12.0,Medium-bodied,Medium,BR,Brazil,1001,Serra Gaúcha,10001,Casa Perini,http://www.vinicolaperini.com.br,"[2016, 2015, 2014, 2013, 2012, 2011, 2010, 200...",3.250000
2,100003,Cabernet Sauvignon,Red,Varietal/100%,['Cabernet Sauvignon'],"['Beef', 'Lamb', 'Poultry']",12.0,Full-bodied,High,BR,Brazil,1001,Serra Gaúcha,10002,Castellamare,https://www.emporiocastellamare.com.br,"[2021, 2020, 2019, 2018, 2017, 2016, 2015, 201...",3.107143
3,100005,Maison de Ville Cabernet-Merlot,Red,Assemblage/Bordeaux Red Blend,"['Cabernet Sauvignon', 'Merlot']","['Beef', 'Lamb', 'Game Meat', 'Poultry']",11.0,Full-bodied,Medium,BR,Brazil,1001,Serra Gaúcha,10000,Aurora,http://www.vinicolaaurora.com.br,"[2021, 2020, 2019, 2018, 2017, 2016, 2015, 201...",2.700000
4,100007,Do Lugar Moscatel Espumantes,Sparkling,Varietal/100%,['Muscat/Moscato Bianco'],"['Pork', 'Rich Fish', 'Shellfish']",7.5,Medium-bodied,High,BR,Brazil,1001,Serra Gaúcha,10012,Dal Pizzol,http://www.dalpizzol.com.br,"[2018, 2017, 2016, 2015, 2014, 2013, 2012, 201...",3.326087
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1002,199408,Petite Arvine,White,Varietal/100%,['Petite Arvine'],"['Pasta', 'Shellfish', 'Maturated Cheese', 'Ha...",13.5,Medium-bodied,High,CH,Switzerland,2251,Valais,66662,Niklaus Wittwer,http://www.niklaus-wittwer.ch,"[2020, 2019, 2018, 2017, 2016, 2015, 2014, 201...",3.833333
1003,199481,St. Laurent Reserve,Red,Varietal/100%,['St. Laurent'],"['Beef', 'Lamb', 'Game Meat']",14.0,Medium-bodied,High,AT,Austria,2079,Mittelburgenland,62148,Hundsdorfer,http://www.hundsdorfer.at,"[2018, 2017, 2016, 2015, 2013, 2012, 2011, 200...",4.000000
1004,199533,Mariengarten Chardonnay,White,Varietal/100%,['Chardonnay'],"['Pork', 'Rich Fish', 'Vegetarian', 'Poultry']",12.2,Full-bodied,High,AT,Austria,2086,Südsteiermark,62556,Muster-Gamlitz,http://www.muster-gamlitz.at,"[2021, 2020, 2019, 2018, 2017, 2016, 2015, 201...",3.166667
1005,199885,Grüner Veltliner Federspiel Ried Kreuzberg,White,Varietal/100%,['Grüner Veltliner'],"['Pork', 'Vegetarian', 'Poultry', 'Lean Fish',...",12.2,Medium-bodied,High,AT,Austria,2089,Wachau,62656,Josef Fischer,http://www.huchenfischer.at,"[2018, 2016, 2015, 2014, 2013, 2012, 2011]",3.666667


In [5]:
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import OneHotEncoder


# Function to find the top-rated wine by a user and recommend similar wines using KNN
def recommend_similar_wines(user_id, merged_data, k=10):
    # Filter the wines rated by the specific user
    user_wines = merged_data[merged_data['UserID'] == user_id]

    if user_wines.empty:
        return f"No wines found for user {user_id}."

    # Find the top-rated wine for this user
    top_rated_wine = user_wines.loc[user_wines['Rating'].idxmax()]

    # Extract relevant features (e.g., 'Type', 'Body') for KNN
    features = ['Type', 'Body']  # Adjust based on your dataset

    # Prepare the dataset for KNN (focus on wine characteristics, exclude 'WineID')
    wine_features = merged_data[features]

    # Apply OneHotEncoding to handle categorical variables like 'Type' and 'Body'
    encoder = OneHotEncoder(sparse_output=False)
    encoded_wine_features = encoder.fit_transform(wine_features)

    # Fit KNN model
    knn_model = NearestNeighbors(n_neighbors=k, metric='euclidean')
    knn_model.fit(encoded_wine_features)

    # Encode the top-rated wine's features (ensure it is passed as a DataFrame with feature names)
    top_rated_wine_features = pd.DataFrame([top_rated_wine[features]], columns=features)
    encoded_top_rated_wine_features = encoder.transform(top_rated_wine_features)

    # Find K nearest wines to the top-rated wine
    distances, indices = knn_model.kneighbors(encoded_top_rated_wine_features)

    # Get the recommended similar wines (excluding the top-rated wine itself)
    recommended_wines = merged_data.iloc[indices[0]]

    # Exclude the top-rated wine itself from the recommendations
    recommended_wines = recommended_wines[recommended_wines['WineID'] != top_rated_wine['WineID']]

    return recommended_wines[['WineID', 'Type', 'Body']]


# Example usage for a specific user (replace with an actual user_id from your dataset)
user_id = 1356810  # Replace with the actual user_id you want to check

# Find the best wine for the user
result = recommend_wine_for_user(user_id, merged_data)
print(result)

# Recommend similar wines based on the top-rated wine
recommended_wines = recommend_similar_wines(user_id, merged_data)

# Display the recommended wines
print(recommended_wines)

(np.int64(112084), np.float64(5.0))
     WineID       Type           Body
25   102547  Sparkling  Medium-bodied
153  116266  Sparkling  Medium-bodied
150  111484  Sparkling  Medium-bodied
64   111927  Sparkling  Medium-bodied
127  111544  Sparkling  Medium-bodied
78   111479  Sparkling  Medium-bodied
37   116266  Sparkling  Medium-bodied
107  111544  Sparkling  Medium-bodied
93   111544  Sparkling  Medium-bodied


In [6]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import OneHotEncoder

# Function to find similar wines using KNN for a given wine
def find_knn_for_wine(wine_id, merged_data, user_id, k=10):
    # Extract relevant features (e.g., 'Type', 'Body') for KNN
    features = ['Type', 'Body']  # Adjust based on your dataset

    # Prepare the dataset for KNN (focus on wine characteristics, exclude 'WineID')
    wine_features = merged_data[features]

    # Apply OneHotEncoding to handle categorical variables like 'Type' and 'Body'
    encoder = OneHotEncoder(sparse_output=False)
    encoded_wine_features = encoder.fit_transform(wine_features)

    # Fit KNN model
    knn_model = NearestNeighbors(n_neighbors=k, metric='euclidean')
    knn_model.fit(encoded_wine_features)

    # Get the features of the wine with the specified wine_id
    target_wine = merged_data[merged_data['WineID'] == wine_id]

    if target_wine.empty:
        return f"No wine found with WineID {wine_id}."

    # Encode the target wine's features (ensure it is passed as a DataFrame with feature names)
    target_wine_features = target_wine[features]
    encoded_target_wine_features = encoder.transform(target_wine_features)

    # Find K nearest wines to the target wine
    distances, indices = knn_model.kneighbors(encoded_target_wine_features)

    # Get the recommended similar wines (excluding the target wine itself)
    recommended_wines = merged_data.iloc[indices[0]]

    # Exclude the target wine itself from the recommendations
    recommended_wines = recommended_wines[recommended_wines['WineID'] != wine_id]

    # Return the recommended wines in a summarized format (list of dicts with WineID, Type, Body, Rating)
    recommended_list = recommended_wines[['WineID', 'Type', 'Body', 'Rating']].to_dict(orient='records')
    
    # get the wine that it eas recommended on
    original_wine = wine_data.where(wine_data['WineID'] == wine_id).dropna()
    original_wine_rating = ratings_data.where((ratings_data['WineID'] == wine_id) & (ratings_data['UserID'] == user_id)).dropna().iloc[0]['Rating']
    recommended_list.append({"WineID": wine_id, "Type": original_wine.iloc[0]["Type"], "Body": original_wine.iloc[0]["Body"], "Rating": original_wine_rating})

    return recommended_list

# Function to find KNN recommendations for all wines in the group recommendation DataFrame
def recommend_similar_wines_for_group(result_df, merged_data, k=10):
    knn_recommendations = []

    # Iterate over each wine recommended to the group
    for index, row in result_df.iterrows():
        wine_id = row['wine_id']
        user_id = row['user_id']

        # Find KNN recommendations for the wine
        knn_result = find_knn_for_wine(wine_id, merged_data, user_id, k)

        # Store the results in a list with the user_id for reference
        knn_recommendations.append({
            'user_id': int(user_id),
            'wine_id': int(wine_id),
            'knn_recommendations': knn_result
        })
    
    # Convert the recommendations into a DataFrame for easy viewing
    knn_recommendation_df = pd.DataFrame(knn_recommendations)
    return knn_recommendation_df

# Example usage (replace 'result_df' with the actual DataFrame from group recommendations)
result_df = recommend_wine_for_group(group_id=1, group_data=group_data, merged_data=merged_data)

# Find KNN recommendations for all wines in the result_df
knn_recommendation_df = recommend_similar_wines_for_group(result_df, merged_data, k=10)

# Display the KNN recommendations for each user's wine
knn_recommendation_df


,user_id,wine_id,knn_recommendations
0,1180876,111521,"[{'WineID': 102547, 'Type': 'Sparkling', 'Body..."
1,1006585,111422,"[{'WineID': 111431, 'Type': 'Red', 'Body': 'Fu..."


In [7]:
# function to expand each row
#def expand_dict_list(row):
#    # for each dictionary in the list, create a new column name with the index and expand the dict
#    expanded_cols = {}
#    for idx, d in enumerate(row):
#        for key, value in d.items():
#            expanded_cols[f'{idx+1}-{key}'] = value
#    return pd.Series(expanded_cols)
#
## apply the function to the last column of the DataFrame
#expanded_df = knn_recommendation_df['knn_recommendations'].apply(expand_dict_list)
#
## drop the original column with the list of dicts and concatenate the expanded DataFrame
#df_final = pd.concat([knn_recommendation_df.drop(columns=['knn_recommendations']), expanded_df], axis=1)
#df_final

In [8]:
# each property gets a score from each user, based on the ordering. Then sum the scores of each characteristic and choose the wine that have those characteristics. If multiple wines, then choose based on the rating (todo)

In [9]:
def score_characteristics(recommendations):
    scores = {"Type": {}, "Body": {}}
    score = len(recommendations)
    c = 0
    for wine in recommendations:
        for key, value in wine.items():
            if key in ["WineID", "Rating"]:
                continue
            if key == "Type":
                if value not in scores["Type"]:
                    scores["Type"][value] = 1
                else:
                    scores["Type"][value] += 1
            elif key == "Body":
                if value not in scores["Body"]:
                    scores["Body"][value] = 1
                else:
                    scores["Body"][value] += 1
        c+=1
    
    return scores

category_weights_by_user = []
for index, row in knn_recommendation_df.iterrows():
    x = score_characteristics(row.knn_recommendations)
    category_weights_by_user.append(x)
    print(x)

{'Type': {'Sparkling': 11}, 'Body': {'Medium-bodied': 11}}
{'Type': {'Red': 11}, 'Body': {'Full-bodied': 11}}


In [10]:
category_weights = {"Type": {}, "Body": {}}
for i in category_weights_by_user:
    for key, value in i["Type"].items():
        if key not in category_weights:
            category_weights["Type"][key] = value
        else:
            category_weights["Type"][key] += value
        
    for key, value in i["Body"].items():
        if key not in category_weights:
            category_weights["Body"][key] = value
        else:
            category_weights["Body"][key] += value

category_weights

{'Type': {'Sparkling': 11, 'Red': 11},
 'Body': {'Medium-bodied': 11, 'Full-bodied': 11}}

In [11]:
# order according to the scores
category_weights_sorted = {"Type": {}, "Body": {}}
for key, value in category_weights.items():
    category_weights_sorted[key] = dict(sorted(category_weights[key].items(), key=lambda item: item[1], reverse=True))

category_weights_sorted

{'Type': {'Sparkling': 11, 'Red': 11},
 'Body': {'Medium-bodied': 11, 'Full-bodied': 11}}

In [14]:
# select wine with top categories of each type
def get_wine_with_top_categories(category_weights_sorted=category_weights_sorted, top_type_index=1, top_body_index=1):
    top_type = list(category_weights_sorted["Type"].keys())[top_type_index-1]
    top_body = list(category_weights_sorted["Body"].keys())[top_body_index-1]
    
    # find a wine with these categories
    selection = wine_data.where((wine_data['Type'] == top_type) & (wine_data['Body'] == top_body)).dropna()
    
    if selection.shape[0] == 0:
        if top_type_index == top_body_index:
            selection = get_wine_with_top_categories(category_weights_sorted=category_weights_sorted, top_type_index=top_type_index+1, top_body_index=top_body_index)
        else:
            selection = get_wine_with_top_categories(category_weights_sorted=category_weights_sorted,top_type_index=top_type_index, top_body_index=top_body_index+1)
    
    selection.sort_values(by='AvgRating', ascending=False, inplace=True)

    return selection.head(1)

get_wine_with_top_categories()

,WineID,WineName,Type,Elaborate,Grapes,Harmonize,ABV,Body,Acidity,Code,Country,RegionID,RegionName,WineryID,WineryName,Website,Vintages,AvgRating
258,112084.0,Oenothèque Brut Champagne,Sparkling,Assemblage/Blend,"['Pinot Noir', 'Chardonnay']","['Pork', 'Rich Fish', 'Shellfish', 'Soft Cheese']",12.5,Medium-bodied,High,FR,France,1188.0,Champagne,21604.0,Dom Pérignon,https://www.domperignon.com,"[2021, 2020, 2019, 2018, 2017, 2016, 2015, 201...",4.757212
